# 🤖 Notebook de inferencia de ventas 2025
Este notebook importa las librerías principales y carga el archivo de ventas para inferencia.

In [85]:
# Importación de librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import streamlit as st
import holidays

In [86]:
# Carga del archivo ventas_2025_inferencia en inferencia_df
inferencia_path = '../data/raw/inferencia/ventas_2025_inferencia.csv'
inferencia_df = pd.read_csv(inferencia_path)

In [87]:
# Visualización de las primeras filas de inferencia_df
display(inferencia_df.head())

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


## 🔄 Transformación de inferencia_df
Aplicando las mismas transformaciones que se hicieron en el notebook de entrenamiento

In [88]:
# 1. Conversión de fecha a datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'], errors='coerce')
print('Fecha convertida a datetime')
print(inferencia_df['fecha'].dtype)

Fecha convertida a datetime
datetime64[ns]


In [89]:
# 2. Variables temporales y de calendario usando holidays para Ecuador
inferencia_df['año'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.dayofweek
inferencia_df['nombre_dia_semana'] = inferencia_df['fecha'].dt.day_name()
inferencia_df['es_fin_de_semana'] = inferencia_df['dia_semana'].isin([5,6])

# Festivos en Ecuador
festivos_ecuador = holidays.country_holidays('EC', years=inferencia_df['año'].unique())
inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(festivos_ecuador)
inferencia_df['nombre_festivo'] = inferencia_df['fecha'].map(festivos_ecuador)

# Black Friday: último viernes de noviembre
def es_black_friday(fecha):
    if fecha.month == 11 and fecha.weekday() == 4:
        ult_viernes = max([d for d in pd.date_range(start=f'{fecha.year}-11-01', end=f'{fecha.year}-11-30') if d.weekday() == 4])
        return fecha == ult_viernes
    return False

inferencia_df['es_black_friday'] = inferencia_df['fecha'].apply(es_black_friday)
inferencia_df['es_cyber_monday'] = inferencia_df['fecha'].apply(lambda x: x.month == 11 and x.weekday() == 0 and 27 <= x.day <= 30)
inferencia_df['semana_año'] = inferencia_df['fecha'].dt.isocalendar().week
inferencia_df['trimestre'] = inferencia_df['fecha'].dt.quarter
inferencia_df['inicio_mes'] = inferencia_df['dia_mes'] <= 7
inferencia_df['fin_mes'] = inferencia_df['dia_mes'] >= (inferencia_df['fecha'] + pd.offsets.MonthEnd(0)).dt.day - 6
inferencia_df['es_primer_dia_mes'] = inferencia_df['dia_mes'] == 1
inferencia_df['es_ultimo_dia_mes'] = inferencia_df['dia_mes'] == (inferencia_df['fecha'] + pd.offsets.MonthEnd(0)).dt.day
inferencia_df['es_dia_laborable'] = (~inferencia_df['es_fin_de_semana']) & (~inferencia_df['es_festivo'])

print('Variables temporales y de calendario creadas')
print(inferencia_df[['fecha','año','mes','dia_mes','es_festivo','es_black_friday']].head())

Variables temporales y de calendario creadas
       fecha   año  mes  dia_mes  es_festivo  es_black_friday
0 2025-10-25  2025   10       25       False            False
1 2025-10-25  2025   10       25       False            False
2 2025-10-25  2025   10       25       False            False
3 2025-10-25  2025   10       25       False            False
4 2025-10-25  2025   10       25       False            False


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_20488\1472800572.py:11: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(festivos_ecuador)


In [90]:
# 3. Crear lags y media móvil de 7 días por producto (SIN agrupar por año en inferencia)
inferencia_df = inferencia_df.sort_values(['producto_id', 'fecha'])
for lag in range(1, 8):
    inferencia_df[f'unidades_vendidas_lag{lag}'] = inferencia_df.groupby('producto_id')['unidades_vendidas'].shift(lag)

inferencia_df['unidades_vendidas_mm7'] = inferencia_df.groupby('producto_id')['unidades_vendidas'].transform(lambda x: x.rolling(window=7, min_periods=7).mean())

print('Lags y media móvil creados')
print(inferencia_df[['fecha','producto_id','unidades_vendidas','unidades_vendidas_lag1','unidades_vendidas_mm7']].head(10))

Lags y media móvil creados
         fecha producto_id  unidades_vendidas  unidades_vendidas_lag1  \
0   2025-10-25    PROD_001               26.0                     NaN   
24  2025-10-26    PROD_001               20.0                    26.0   
48  2025-10-27    PROD_001               16.0                    20.0   
72  2025-10-28    PROD_001               15.0                    16.0   
96  2025-10-29    PROD_001               14.0                    15.0   
120 2025-10-30    PROD_001               10.0                    14.0   
144 2025-10-31    PROD_001               14.0                    10.0   
168 2025-11-01    PROD_001                NaN                    14.0   
192 2025-11-02    PROD_001                NaN                     NaN   
216 2025-11-03    PROD_001                NaN                     NaN   

     unidades_vendidas_mm7  
0                      NaN  
24                     NaN  
48                     NaN  
72                     NaN  
96                     N

In [91]:
# 4. Variable descuento_porcentaje
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100
print('Variable descuento_porcentaje creada')
print(inferencia_df[['precio_base', 'precio_venta', 'descuento_porcentaje']].head())

Variable descuento_porcentaje creada
    precio_base  precio_venta  descuento_porcentaje
0           115        113.13             -1.626087
24          115        105.75             -8.043478
48          115        114.95             -0.043478
72          115        117.31              2.008696
96          115        108.10             -6.000000


In [92]:
# Verificación de columnas necesarias
print('Columnas actuales en inferencia_df:')
print(inferencia_df.columns.tolist())
print(f'\n¿Existen columnas de competencia? Amazon: {"Amazon" in inferencia_df.columns}, Decathlon: {"Decathlon" in inferencia_df.columns}, Deporvillage: {"Deporvillage" in inferencia_df.columns}')

Columnas actuales en inferencia_df:
['fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria', 'precio_base', 'es_estrella', 'unidades_vendidas', 'precio_venta', 'ingresos', 'Amazon', 'Decathlon', 'Deporvillage', 'año', 'mes', 'dia_mes', 'dia_semana', 'nombre_dia_semana', 'es_fin_de_semana', 'es_festivo', 'nombre_festivo', 'es_black_friday', 'es_cyber_monday', 'semana_año', 'trimestre', 'inicio_mes', 'fin_mes', 'es_primer_dia_mes', 'es_ultimo_dia_mes', 'es_dia_laborable', 'unidades_vendidas_lag1', 'unidades_vendidas_lag2', 'unidades_vendidas_lag3', 'unidades_vendidas_lag4', 'unidades_vendidas_lag5', 'unidades_vendidas_lag6', 'unidades_vendidas_lag7', 'unidades_vendidas_mm7', 'descuento_porcentaje']

¿Existen columnas de competencia? Amazon: True, Decathlon: True, Deporvillage: True


In [93]:
# 5. Crear variable precio_competencia y ratio_precio
inferencia_df['precio_competencia'] = inferencia_df[['Amazon', 'Decathlon', 'Deporvillage']].mean(axis=1)
inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']

# Eliminar variables individuales de competidores
inferencia_df = inferencia_df.drop(columns=['Amazon', 'Decathlon', 'Deporvillage'])
print('Variables precio_competencia y ratio_precio creadas, columnas de competidores eliminadas')
print(inferencia_df[['precio_venta', 'precio_competencia', 'ratio_precio']].head())

Variables precio_competencia y ratio_precio creadas, columnas de competidores eliminadas
    precio_venta  precio_competencia  ratio_precio
0         113.13          102.573333      1.102918
24        105.75           98.356667      1.075169
48        114.95           97.740000      1.176079
72        117.31          103.146667      1.137313
96        108.10          100.520000      1.075408


In [94]:
# 6. One-hot encoding de nombre, categoria y subcategoria
inferencia_df['nombre_h'] = inferencia_df['nombre']
inferencia_df['categoria_h'] = inferencia_df['categoria']
inferencia_df['subcategoria_h'] = inferencia_df['subcategoria']

inferencia_df = pd.get_dummies(inferencia_df, columns=['nombre_h', 'categoria_h', 'subcategoria_h'], drop_first=False)
print('Variables one-hot creadas')
print(f'Total de columnas después de one-hot: {inferencia_df.shape[1]}')

Variables one-hot creadas
Total de columnas después de one-hot: 82


In [95]:
inferencia_df.producto_id.nunique()

24

In [96]:
# 7. Verificar nulls en lags (solo para diagnóstico, NO eliminamos aún)
vars_lags = [f'unidades_vendidas_lag{lag}' for lag in range(1,8)] + ['unidades_vendidas_mm7']
print(f'Total de registros: {len(inferencia_df)}')
print(f'\nNulls por variable lag:')
for var in vars_lags:
    nulls = inferencia_df[var].isnull().sum()
    print(f'  {var}: {nulls} nulls')

print(f'\nDistribución de nulls por mes:')
print(inferencia_df.groupby('mes')[vars_lags].apply(lambda x: x.isnull().sum().sum()))

Total de registros: 888

Nulls por variable lag:
  unidades_vendidas_lag1: 720 nulls
  unidades_vendidas_lag2: 720 nulls
  unidades_vendidas_lag3: 720 nulls
  unidades_vendidas_lag4: 720 nulls
  unidades_vendidas_lag5: 720 nulls
  unidades_vendidas_lag6: 720 nulls
  unidades_vendidas_lag7: 720 nulls
  unidades_vendidas_mm7: 864 nulls

Distribución de nulls por mes:
mes
10     816
11    5088
dtype: int64


In [97]:
# 8. ELIMINAR OCTUBRE COMPLETO y mantener solo NOVIEMBRE (sin importar nulls)
print(f'Registros antes de filtrar por mes: {len(inferencia_df)}')
print(f'Distribución por mes antes del filtro:')
print(inferencia_df['mes'].value_counts().sort_index())

# Filtrar solo noviembre
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()

print(f'\n✅ Registros finales (solo noviembre): {len(inferencia_df)}')
print(f'Rango de fechas final: {inferencia_df["fecha"].min()} a {inferencia_df["fecha"].max()}')
print(f'\nProductos únicos en noviembre: {inferencia_df["producto_id"].nunique()}')
print(f'Días únicos en noviembre: {inferencia_df["fecha"].nunique()}')

Registros antes de filtrar por mes: 888
Distribución por mes antes del filtro:
mes
10    168
11    720
Name: count, dtype: int64

✅ Registros finales (solo noviembre): 720
Rango de fechas final: 2025-11-01 00:00:00 a 2025-11-30 00:00:00

Productos únicos en noviembre: 24
Días únicos en noviembre: 30


In [98]:
# 9. Verificación de nulls en noviembre (después de eliminar octubre)
print('🔍 Verificando nulls en las variables lag después de filtrar noviembre:')
vars_lags = [f'unidades_vendidas_lag{lag}' for lag in range(1,8)] + ['unidades_vendidas_mm7']
nulls_resumen = inferencia_df[vars_lags].isnull().sum()
print(nulls_resumen)

if nulls_resumen.sum() > 0:
    print(f'\n⚠️ Hay {nulls_resumen.sum()} valores null en variables lag en noviembre.')
    print('Esto es NORMAL para los primeros días de noviembre que usan lags de octubre.')
    print('\nPrimeras fechas con nulls:')
    df_con_nulls = inferencia_df[inferencia_df[vars_lags].isnull().any(axis=1)]
    print(df_con_nulls[['fecha', 'producto_id'] + vars_lags].head(10))
else:
    print('✅ No hay nulls en variables lag en noviembre.')

🔍 Verificando nulls en las variables lag después de filtrar noviembre:
unidades_vendidas_lag1    696
unidades_vendidas_lag2    672
unidades_vendidas_lag3    648
unidades_vendidas_lag4    624
unidades_vendidas_lag5    600
unidades_vendidas_lag6    576
unidades_vendidas_lag7    552
unidades_vendidas_mm7     720
dtype: int64

⚠️ Hay 5088 valores null en variables lag en noviembre.
Esto es NORMAL para los primeros días de noviembre que usan lags de octubre.

Primeras fechas con nulls:
         fecha producto_id  unidades_vendidas_lag1  unidades_vendidas_lag2  \
168 2025-11-01    PROD_001                    14.0                    10.0   
192 2025-11-02    PROD_001                     NaN                    14.0   
216 2025-11-03    PROD_001                     NaN                     NaN   
240 2025-11-04    PROD_001                     NaN                     NaN   
264 2025-11-05    PROD_001                     NaN                     NaN   
288 2025-11-06    PROD_001                    

In [99]:
# 10. Rellenar nulls en lags con estrategia hacia adelante (forward fill por producto)
# Los primeros días de noviembre que tienen nulls en lags usarán el valor más reciente disponible
print('🔧 Rellenando nulls en variables lag...')
vars_lags = [f'unidades_vendidas_lag{lag}' for lag in range(1,8)] + ['unidades_vendidas_mm7']

for var in vars_lags:
    nulls_antes = inferencia_df[var].isnull().sum()
    if nulls_antes > 0:
        # Forward fill por producto (usa el último valor válido de octubre)
        inferencia_df[var] = inferencia_df.groupby('producto_id')[var].fillna(method='ffill')
        # Si aún quedan nulls (primeros días sin histórico), usar media del producto
        inferencia_df[var] = inferencia_df.groupby('producto_id')[var].transform(lambda x: x.fillna(x.mean()))
        nulls_despues = inferencia_df[var].isnull().sum()
        print(f'  {var}: {nulls_antes} nulls → {nulls_despues} nulls')

print(f'\n✅ Nulls rellenados. Verificación final de nulls en todo el dataframe:')
print(inferencia_df[vars_lags].isnull().sum().sum())

🔧 Rellenando nulls en variables lag...
  unidades_vendidas_lag1: 696 nulls → 0 nulls
  unidades_vendidas_lag2: 672 nulls → 0 nulls
  unidades_vendidas_lag3: 648 nulls → 0 nulls
  unidades_vendidas_lag4: 624 nulls → 0 nulls
  unidades_vendidas_lag5: 600 nulls → 0 nulls
  unidades_vendidas_lag6: 576 nulls → 0 nulls
  unidades_vendidas_lag7: 552 nulls → 0 nulls
  unidades_vendidas_mm7: 720 nulls → 720 nulls

✅ Nulls rellenados. Verificación final de nulls en todo el dataframe:
720
  unidades_vendidas_lag7: 552 nulls → 0 nulls
  unidades_vendidas_mm7: 720 nulls → 720 nulls

✅ Nulls rellenados. Verificación final de nulls en todo el dataframe:
720


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_20488\1236518608.py:10: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  inferencia_df[var] = inferencia_df.groupby('producto_id')[var].fillna(method='ffill')
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_20488\1236518608.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  inferencia_df[var] = inferencia_df.groupby('producto_id')[var].fillna(method='ffill')
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_20488\1236518608.py:10: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  inferencia_df[var] 

In [100]:
# 11. Guardar dataframe transformado
inferencia_df.to_csv('../data/processed/inferencia_df_transformado.csv', index=False)
print('✅ DataFrame transformado guardado en data/processed/inferencia_df_transformado.csv')

# Verificación final
print(f'\n📊 RESUMEN FINAL:')
print(f'Shape final: {inferencia_df.shape}')
print(f'Productos únicos: {inferencia_df["producto_id"].nunique()}')
print(f'Rango de fechas: {inferencia_df["fecha"].min()} a {inferencia_df["fecha"].max()}')
print(f'Total de días: {inferencia_df["fecha"].nunique()}')
print(f'Registros esperados (24 productos × 30 días): {24 * 30} = 720')
print(f'Registros reales: {len(inferencia_df)}')
print(f'\nColumnas totales: {len(inferencia_df.columns)}')
display(inferencia_df.head(10))
display(inferencia_df.tail(10))

✅ DataFrame transformado guardado en data/processed/inferencia_df_transformado.csv

📊 RESUMEN FINAL:
Shape final: (720, 82)
Productos únicos: 24
Rango de fechas: 2025-11-01 00:00:00 a 2025-11-30 00:00:00
Total de días: 30
Registros esperados (24 productos × 30 días): 720 = 720
Registros reales: 720

Columnas totales: 82


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,...,subcategoria_h_Esterilla Yoga,subcategoria_h_Mancuernas Ajustables,subcategoria_h_Mochila Trekking,subcategoria_h_Pesa Rusa,subcategoria_h_Pesas Casa,subcategoria_h_Rodillera Yoga,subcategoria_h_Ropa Montaña,subcategoria_h_Ropa Running,subcategoria_h_Zapatillas Running,subcategoria_h_Zapatillas Trail
168,2025-11-01,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
192,2025-11-02,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
216,2025-11-03,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
240,2025-11-04,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
264,2025-11-05,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
288,2025-11-06,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
312,2025-11-07,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
336,2025-11-08,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
360,2025-11-09,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False
384,2025-11-10,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,False,False,False,False,False,False,False,False,True,False


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,...,subcategoria_h_Esterilla Yoga,subcategoria_h_Mancuernas Ajustables,subcategoria_h_Mochila Trekking,subcategoria_h_Pesa Rusa,subcategoria_h_Pesas Casa,subcategoria_h_Rodillera Yoga,subcategoria_h_Ropa Montaña,subcategoria_h_Ropa Running,subcategoria_h_Zapatillas Running,subcategoria_h_Zapatillas Trail
671,2025-11-21,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,50.08,NaN,...,False,False,False,False,False,False,False,False,False,False
695,2025-11-22,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,50.57,NaN,...,False,False,False,False,False,False,False,False,False,False
719,2025-11-23,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,49.50,NaN,...,False,False,False,False,False,False,False,False,False,False
743,2025-11-24,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,50.87,NaN,...,False,False,False,False,False,False,False,False,False,False
767,2025-11-25,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,49.04,NaN,...,False,False,False,False,False,False,False,False,False,False
791,2025-11-26,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,50.85,NaN,...,False,False,False,False,False,False,False,False,False,False
815,2025-11-27,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,49.82,NaN,...,False,False,False,False,False,False,False,False,False,False
839,2025-11-28,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,42.50,NaN,...,False,False,False,False,False,False,False,False,False,False
863,2025-11-29,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,50.37,NaN,...,False,False,False,False,False,False,False,False,False,False
887,2025-11-30,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,49.62,NaN,...,False,False,False,False,False,False,False,False,False,False


In [103]:
inferencia_df.columns

Index(['fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria',
       'precio_base', 'es_estrella', 'unidades_vendidas', 'precio_venta',
       'ingresos', 'año', 'mes', 'dia_mes', 'dia_semana', 'nombre_dia_semana',
       'es_fin_de_semana', 'es_festivo', 'nombre_festivo', 'es_black_friday',
       'es_cyber_monday', 'semana_año', 'trimestre', 'inicio_mes', 'fin_mes',
       'es_primer_dia_mes', 'es_ultimo_dia_mes', 'es_dia_laborable',
       'unidades_vendidas_lag1', 'unidades_vendidas_lag2',
       'unidades_vendidas_lag3', 'unidades_vendidas_lag4',
       'unidades_vendidas_lag5', 'unidades_vendidas_lag6',
       'unidades_vendidas_lag7', 'unidades_vendidas_mm7',
       'descuento_porcentaje', 'precio_competencia', 'ratio_precio',
       'nombre_h_Adidas Own The Run Jacket', 'nombre_h_Adidas Ultraboost 23',
       'nombre_h_Asics Gel Nimbus 25', 'nombre_h_Bowflex SelectTech 552',
       'nombre_h_Columbia Silver Ridge',
       'nombre_h_Decathlon Bandas Elásticas Set', 'nomb